In [1]:
import psycopg2
import pandas as pd

In [2]:
dbname = 'podcast'
username = 'lindsay'

In [3]:
podcastDf = pd.read_pickle('pkl/clean_podcast_data.pkl')

In [4]:
con = psycopg2.connect(database = dbname, user = username)

In [6]:
cursor = con.cursor()

In [7]:
thisPod = podcastDf.iloc[0]
thisPod

artist_id                                                                  NaN
artist_name                                     David Chen and Joanna Robinson
artist_view_url                                                            NaN
artwork_url100               http://is1.mzstatic.com/image/thumb/Music/v4/0...
artwork_url30                http://is1.mzstatic.com/image/thumb/Music/v4/0...
artwork_url60                http://is1.mzstatic.com/image/thumb/Music/v4/0...
artwork_url600               http://is1.mzstatic.com/image/thumb/Music/v4/0...
collection_censored_name           A Cast of Kings - A Game of Thrones Podcast
collection_explicitness                                               explicit
collection_hd_price                                                          0
collection_id                                                        515836681
collection_name                    A Cast of Kings - A Game of Thrones Podcast
collection_price                                    

In [10]:
# insert podcast data into podcast table
query = "INSERT INTO podcast (artwork_url30, artwork_url60, artwork_url100, artwork_url600, explicit, name, view_url, summary) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

for ind, thisPod in podcastDf.iterrows():
    data = (thisPod['artwork_url30'], thisPod['artwork_url60'], thisPod['artwork_url100'], thisPod['artwork_url600'], thisPod['content_advisory_rating'], thisPod['collection_censored_name'], thisPod['collection_view_url'], thisPod['podcast_summary'])
    cursor.execute(query, data)
con.commit()

In [18]:
# get unique artists
artistDf = podcastDf[['artist_name', 'artist_view_url']]
artistDf = artistDf.drop_duplicates()
artistDf.shape

(4863, 2)

In [20]:
# insert artist data into artist table
query = "INSERT INTO artist (view_url, name) VALUES (%s, %s)"

for ind, row in artistDf.iterrows():
    data = (row['artist_view_url'], row['artist_name'])
    cursor.execute(query, data)
con.commit()

In [48]:
# get unique genres
genreDf = podcastDf['genres']
genreList = []
for row in genreDf:
    for genre in row:
        genreList.append(genre)
genreList = list(set(genreList))

In [66]:
# insert genre data into genre table
query = "INSERT INTO genre (name) VALUES (%s);"
for item in genreList:
    data = (item, )
    cursor.execute(query, data)
con.commit()